In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
import sys
import os
import torch
from functools import partial
from torch.utils.data import Dataset, DataLoader

import torch.nn as nn
from embedding import CustomEmbedding
from transformers import EncoderDecoderTransformer
from utils import padding_collate_fn

from generate_data import RandomIntegerDataset

In [45]:
n_real_tokens = 10
PAD_TOKEN_IDX = n_real_tokens
SOS_TOKEN_IDX = n_real_tokens + 1
EOS_TOKEN_IDX = n_real_tokens + 2
vocab_size = n_real_tokens + 3
D_MODEL = 16

embeddings = CustomEmbedding(vocab_size = vocab_size + 3, d_model = D_MODEL) # 3 = PAD, SOS, EOS

indices = torch.tensor([1,9])

# print(embeddings.embeddings.weight)
print(embeddings.embeddings(indices))

tensor([[ 0.8193, -0.5992,  0.1428,  0.0840,  1.0485, -0.8220,  2.3278,  0.9852,
          0.0750, -0.3915,  1.4703,  0.9527,  0.2241,  0.2545,  0.8176,  0.1739],
        [ 0.9532,  0.5967, -0.6777,  0.8112,  0.4193, -1.0345, -0.1871, -1.1249,
          1.8566,  0.2559, -0.0024,  0.9051,  0.3557,  0.2942, -1.7069,  0.7757]],
       grad_fn=<EmbeddingBackward0>)


In [46]:
MAX_CONTEXT_WINDOW = 50

BATCH_SIZE = 2
MIN_SEQ_LEN = 2
MAX_SEQ_LEN = min(10, MAX_CONTEXT_WINDOW)

rand_ds = RandomIntegerDataset(2, 10, 10, [i for i in range(10)])
dataloader = DataLoader(rand_ds, batch_size = BATCH_SIZE, shuffle = True, collate_fn = partial(padding_collate_fn, pad_token_idx = PAD_TOKEN_IDX))

In [47]:
input, label = next(iter(dataloader))
print(input[0])
print(input[1])
print(label)

tensor([[ 4,  5,  5, 10, 10, 10, 10],
        [ 4,  7,  0,  4,  5,  7,  5]])
tensor([[11,  4,  5,  5, 10, 10, 10, 10],
        [11,  0,  4,  4,  5,  5,  7,  7]])
tensor([[ 4,  5,  5, 12, 10, 10, 10, 10],
        [ 0,  4,  4,  5,  5,  7,  7, 12]])


In [48]:
loss_fn = nn.CrossEntropyLoss(ignore_index = PAD_TOKEN_IDX, reduction = 'sum')

model = EncoderDecoderTransformer(
                    embeddings = embeddings, 
                    vocab_size = vocab_size, 
                    d_model = D_MODEL, 
                    num_attention_heads = 4, 
                    num_encoder_layers = 1, 
                    num_decoder_layers = 1, 
                    dim_feedforward = 32, 
                    max_context_window = MAX_CONTEXT_WINDOW)

optim = torch.optim.SGD(params = model.parameters(), lr = 1e-3, momentum = 0.9, weight_decay = 1e-4)

In [ ]:
def run_train_epoch(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module, optimizer: torch.optim.Optimizer):
    
    model.train()

    epoch_loss = 0.0

    for i, ((source, target), label) in enumerate(dataloader):
        print(f'Processing Batch {i+1}...')

        # FORWARD
        pred_logits = model(source, target)
        # CEL only takes 2D predictions (n_batch * seq_len, vocab_size) and 1D labels (n_batch * seq_len)
        batch_loss = loss_fn(pred_logits.view(-1, vocab_size), label.view(-1))

        # LOG
        epoch_loss += batch_loss.item()

        # BACKWARD
        batch_loss.backward()

        # OPTIMIZE
        optimizer.step()
        optimizer.zero_grad()

    average_epoch_loss = epoch_loss / len(dataloader.dataset)

    return average_epoch_loss

def run_test_loop(dataloader: DataLoader, model: nn.Module, loss_fn: nn.Module):
    
    model.eval()

In [50]:
EPOCHS = 1

training_losses = list()

for i in range(EPOCHS):

    training_loss = run_train_epoch(dataloader, model, loss_fn, optim)
    training_losses.append(training_loss)

Processing Batch 1...
Processing Batch 2...
Processing Batch 3...
Processing Batch 4...
Processing Batch 5...
